<a href="https://colab.research.google.com/github/michaelchyan/MSDS422/blob/main/Module_6_Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 5: Random Forest, Gradient Boosted, Extra Trees

### Management/Research Question
### In layman’s terms, what is the management/research question of interest, and why would anyone care?
The question of interest is which companies are going into bankruptcy. It would be very easy to extend the question to also ask which features are most important when it comes to determining risk of bankruptcy. This information could be very helpful in determining a company's financial health and whether or not they are at risk of bankruptcy and what actions should be taken to improve those features of the company.

## Ingest

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib


from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import sklearn


In [115]:
train = pd.read_csv("train.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
test = pd.read_csv("test.csv")
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Last five rows

In [117]:
train.tail()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41999,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


*What are the columns?*

In [118]:
train.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

*What is the shape:  i.e. rows,columns?*

In [119]:
train.shape

(42000, 785)

## EDA

In [120]:
train.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


## Modeling

In [121]:
X=train.drop(labels=['label'], axis=1)
y=train['label']

### Random Forest Classifier

In [122]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfc = RandomForestClassifier(n_estimators = 100, n_jobs=1, criterion="gini")

start = datetime.now()

rfc.fit(X, y)
rfc_result = rfc.predict(test)

end = datetime.now()
print("Random Forest Classifier Runtime:", end-start)

Random Forest Classifier Runtime: 0:00:31.848593


In [123]:
np.savetxt('rfc_result.csv', 
           np.c_[range(1,len(test)+1), rfc_result], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt = '%d')

### PCA Random Forest

In [124]:
df = pd.concat([X, test])


In [125]:
X.shape

(42000, 784)

In [126]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)

start = datetime.now()

pca.fit(df)

end = datetime.now()
print("PCA Runtime:", end-start)

PCA Runtime: 0:00:06.964339


In [127]:
transform = pca.transform(df)


In [128]:
pca_rcf= RandomForestClassifier()

start = datetime.now()

pca_rcf.fit(transform[0:42000], y)

end = datetime.now()
print("PCA fit Runtime:", end-start)

PCA fit Runtime: 0:01:14.831027


In [129]:
transform_test = transform[42000:]
pca_results = pca_rcf.predict(transform_test)


In [130]:
np.savetxt('pca_rfc_result.csv', 
           np.c_[range(1,len(test)+1), pca_results], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt = '%d')

### K-Means Clustering

In [131]:
from sklearn.cluster import MiniBatchKMeans

n_digits = len(np.unique(y))
print(n_digits)
# Initialize KMeans model
kmeans = MiniBatchKMeans(n_clusters = n_digits)
# Fit the model to the training data
kmeans.fit(X)
kmeans.labels_

10


array([7, 6, 3, ..., 9, 1, 4], dtype=int32)

In [132]:
kmeans_result = kmeans.predict(test)

In [133]:
np.savetxt('kmeans_result.csv', 
           np.c_[range(1,len(test)+1), kmeans_result], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt = '%d')

### Fixed Experiment

The problem was that we combined the training and test data when training the model so overfitting was very likely in the result.

In [144]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)

start = datetime.now()

pca.fit(X)

end = datetime.now()
print("PCA Runtime:", end-start)

PCA Runtime: 0:00:04.111739


In [145]:
transform = pca.transform(X)


In [146]:
pca_rcf= RandomForestClassifier()

start = datetime.now()

pca_rcf.fit(transform, y)

end = datetime.now()
print("PCA fit Runtime:", end-start)

PCA fit Runtime: 0:01:13.842888


In [149]:
pca_results = pca_rcf.predict(test)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


ValueError: ignored

In [ ]:
np.savetxt('pca_rfc_result.csv', 
           np.c_[range(1,len(test)+1), pca_results], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt = '%d')

## Conclusion
